In [ ]:
library(data.table) # to handle the data in a more convenient manner
library(tidyverse) # for a better work flow and more tools to wrangle and visualize the data
library(C50) # for C5.0 decision tree algorithm
library(gmodels) # for model evaluation
library(plotly)
library(tree) # for improved decision trees
library(listviewer) # for navigating nested/list objects
library(scales) # for formatting numbers
library(IRdisplay) # to help pretty print tables
options(warn = -1) # for suppressing messages

In [ ]:
loan <- fread("../data/csv/loan_final313.csv", stringsAsFactors=TRUE)

In [ ]:
str(loan)

In [ ]:
summary(loan)

In [ ]:
loan.dt<-as.data.table(loan)
sapply(loan.dt, function(x) length(unique(x)))
a <- as.data.table(sapply(loan.dt, function(x) length(unique(x))))
b <- cbind(names(loan.dt),a) 
colnames(b)<- c("name","count")
loan.dt[,b[count==1,name]:=NULL]
loan.dt[,id:=NULL]
loan.dt[,c("home_ownership_cat","income_cat","term_cat","purpose_cat","interest_payment_cat",
        "loan_condition_cat","grade_cat"):=NULL]
loan.dt[,c("issue_d","final_d"):=NULL]

In [ ]:
loan_num <- loan %>% purrr::keep(is.numeric)
summaries <- loan_num %>%
  summary() %>% # get statistical summaries
  apply(1, function(x) stringr::str_extract(x, "(?<=:).+") %>% as.numeric) %>%
  magrittr::set_colnames(names(summary(1))) %>% # set column names
  magrittr::set_rownames(names(loan_num)) # set row names

summaries

In [ ]:
plotly::plot_ly(loan, x = ~loan_condition, type = "histogram")

In [ ]:
loan.dt[,!"loan_condition"] %>% # select columns
  tidyr::gather() %>% # reshape into long format in columns "key" and "value"
  ggplot(aes(value)) + # plot value
  facet_wrap(~ key, scale = "free" ) + # divide into separate plots by key
  geom_density(fill = "green")  # get density plots

In [ ]:
loan.dt1<-loan.dt %>% purrr::keep(is.numeric)

In [ ]:
cor(loan.dt1) %>%
  corrplot::corrplot.mixed(upper = "ellipse",
                           lower = "number",
                           tl.pos = "lt",
                           number.cex = .5,
                           lower.col = "blue",
                           tl.cex = 0.7)

In [ ]:
set.seed(1903)
train <- loan.dt[ ,sample(.I, .N * 3/4)]
fitc1 <- C50::C5.0(loan_condition~ ., data = loan.dt[train])
fitc <- C50::C5.0(loan.dt[train, -"loan_condition"], loan.dt[train,loan_condition])
fitc

In [ ]:
loan_train <- loan.dt[train]
loan_test <- loan.dt[-train]

In [ ]:
plot(fitc1)

In [ ]:
summary(fitc1)

In [ ]:
fitc_rules <- C50::C5.0(  loan_condition ~., 
                         data = loan.dt[train ,],
                         rules = TRUE )
summary(fitc_rules)

In [ ]:
predc_train <- predict(fitc,
                       newdata = loan.dt[train],
                       type = "class")

In [ ]:
dt_ct6 <- gmodels::CrossTable(loan.dt[train, loan_condition],
                              predc_train,
                              prop.chisq = FALSE,
                              prop.c = FALSE,
                              prop.r = FALSE,
                              dnn = c('actual default', 'predicted default'))

In [ ]:
predc <- predict(fitc,
                 newdata = loan.dt[-train],
                 type = "class")

In [ ]:
dt_ct7 <- gmodels::CrossTable(loan.dt[-train, loan_condition],
                              predc,
                              prop.chisq = FALSE,
                              prop.c = FALSE,
                              prop.r = FALSE,
                              dnn = c('actual default', 'predicted default'))

In [ ]:
fit <- tree::tree( loan_condition ~.,
                  data = loan.dt[train],
                  split = "deviance" )
plot(fit)
text(fit)

In [ ]:
summary(fit)

In [ ]:
pred <- predict(fit,
                newdata = loan.dt[-train])
pred

In [ ]:
pred_class <- colnames(pred)[max.col(pred,
                                     ties.method = c("random"))]

In [ ]:
dt_ct8 <- gmodels::CrossTable(loan.dt[-train, loan_condition],
                              pred_class,
                              prop.chisq = FALSE,
                              prop.c = FALSE,
                              prop.r = FALSE,
                              dnn = c('actual default', 'predicted default'))

In [ ]:
loan_boost10 <- C5.0(loan_train[,-"loan_condition"],
                       loan_train$loan_condition,
                       trials = 10)

In [ ]:
loan_boost10

In [ ]:
summary(loan_boost10)

In [ ]:
loan_result_boost10 <- predict(loan_boost10, loan_train)

In [ ]:
ct_dt3 <- gmodels::CrossTable(loan_train$loan_condition, loan_result_boost10, prop.chisq = F, prop.c = F, prop.r = F,
dnn = c('actual default', 'predicted default'))

ct_dt3

In [ ]:
ct_dt3$prop.tbl %>%
    diag() %>%
    sum() %>%
    scales::percent(accuracy = 0.01)

In [ ]:
loan_pred_boost10 <- predict(loan_boost10, loan_test)

In [ ]:
ct_dt4 <- gmodels::CrossTable(loan_test$loan_condition, loan_pred_boost10, prop.chisq = F, prop.c = F, prop.r = F,
dnn = c('actual default', 'predicted default'))

In [ ]:
ct_dt4

In [ ]:
ct_dt4$t %>% knitr::kable() %>% as.character() %>% IRdisplay::display_html()

In [ ]:
ct_dt4$prop.tbl %>%
    diag() %>%
    sum() %>%
    scales::percent(accuracy = 0.01)